# EndSARS Twitter Analysis


Sentiment and Network Analysis of the #EndSARS protest movement that occured in 2020 in Nigeria. 

## SetUp

### Import Python Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import yaml
import os
import json
import re
import ast
import csv
import gensim
from searchtweets import ResultStream, gen_request_parameters, load_credentials, collect_results
# import fastText as ft
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime, timedelta


%matplotlib inline
pd.set_option("display.max_rows", 500)

C:\Users\USER\anaconda3\envs\endsarsproject\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Connecting to Twitter

I suggest you use Twitter's official searchtweets api which is easy to use and reliable.
<br>In order to get past the tweets retrieval limit, you might also need to  use the premium search api.

- Create a hidden yaml file (.yaml) which contains details about your twitter api keys
- Create a variable that stores the twitter api credentials from the hiddem yaml file above.

**Keywords**: EndSARS, Soro Soke, Police Brutality, lekki toll gate, anti-robbery, lekki massacre, End bad governance,
End swat, buhari is a bad boy, sanwa-olu is a murderer, #LekkiMassacre

Helpful links:
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
- https://github.com/twitterdev/search-tweets-python/tree/v2#using-the-twitter-search-apis-python-wrapper

In [58]:
search_args= load_credentials("tweetkeys1.yaml",
                                       yaml_key="search_tweets_v2",
                                       env_overwrite=False)


In [6]:
keywords= "#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet"
tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
user_fields="id,created_at,location,username,verified,public_metrics"
place_fields="country,country_code,full_name,geo,id,name,place_type"
max_tweets=6000
results_per_call=100
Dates = ['2020-01-01','2020-01-08','2020-01-15','2020-01-22','2020-01-29','2020-02-05','2020-02-12','2020-02-19','2020-02-26', '2020-03-03', '2020-03-09', '2020-03-16', '2020-03-23','2020-03-30','2020-10-13','2020-10-21']







In [7]:
def get_time(start_time, interval = timedelta(days = 7)):
    return (datetime.strptime(start_time, '%Y-%m-%d') + interval).strftime('%Y-%m-%d')


In [8]:
def get_tweets(keywords, start_time, user_fields, place_fields,
               tweet_fields, expansion_fields, results_per_call, max_tweets, result_stream_args):
    query = gen_request_parameters(query = keywords, start_time = start_time, end_time = get_time(start_time),
                                   user_fields = user_fields, place_fields = place_fields,
                                   tweet_fields = tweet_fields, expansions = expansion_fields, results_per_call = results_per_call)
    tweets = collect_results(query, max_tweets = max_tweets, result_stream_args = result_stream_args)
    return tweets





In [9]:
def dataframe(keywords, start_time, user_fields, place_fields, expansion_fields,
              tweet_fields, results_per_call, max_tweets, result_stream_args):
    tweets = list()
    tweets.extend(get_tweets(keywords = keywords, start_time = start_time, tweet_fields = tweet_fields,
                             user_fields = user_fields, place_fields = place_fields, expansion_fields = expansion_fields,
                             results_per_call = results_per_call, max_tweets = max_tweets,
                             result_stream_args = result_stream_args))
    return pd.DataFrame(tweets)

In [10]:
import time
df = pd.DataFrame()
for date in Dates:
    tweets = dataframe(keywords = '#EndSars OR #SoroSoke OR #EndPoliceBrutality OR lekki toll gate OR anti-robbery OR #LekkiMassacre OR End bad governance OR #EndSwat OR buhari is a bad boy OR sanwo-olu is a murderer lang:en -is:retweet',
                   user_fields = 'username,verified,description,location,created_at,id,public_metrics',
                   place_fields = 'country,country_code,full_name,geo,id,name,place_type',
                   expansion_fields = 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
                   tweet_fields = 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive',
                   start_time = date, results_per_call = 100, max_tweets = 6000, result_stream_args = search_args)
    df = pd.concat([df, tweets])
    time.sleep(5)

 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': '#EndSars OR #SoroSoke OR #EndPoliceBrutality OR lekki toll gate OR anti-robbery OR #LekkiMassacre OR End bad governance OR #EndSwat OR buhari is a bad boy OR sanwo-olu is a murderer lang:en -is:retweet', 'max_results': 100, 'start_time': '2020-02-12T00:00:00Z', 'end_time': '2020-02-19T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive', 'user.fields': 'username,verified,description,location,created_at,id,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3fo71grgvsjf9gmf8wm1u9n229mw9z1'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: Rate limit exceeded
 | Too M

In [116]:
df.to_pickle("tweets.pkl")


In [60]:
df=pd.read_pickle('tweets.pkl')

In [61]:
df.tail()

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
6125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1321236368631234560,95.0,b26v89c19zqg8o3fosbsk36ui1eqr4y7yqsp5ohqjg6m5,NaN,NaN,NaN,NaN,NaN,None,None


In [8]:
df.shape

(59738, 25)

In [9]:
search_args  ['bearer_token']


'AAAAAAAAAAAAAAAAAAAAANE2NAEAAAAAS7YKh9sTQnSUY8ERjYLRiASf7Gw%3DOphmA1rPK5Xtc7BQcLAL9GRRas6mTFvEq7kd5U18ydBBsXj5M6'

In [26]:
users= list(df['author_id'])
users
            
#.dropna().unique()) 


['782957256434409472',
 '827863696416321537',
 '3344738026',
 '954711989842010112',
 '313725796',
 '1014823661063344129',
 '326132860',
 '1066054522517114880',
 '398584109',
 '2232504074',
 '1053931388037201920',
 '2387886752',
 '290453376',
 '175055258',
 '190253305',
 '361433351',
 '2600366343',
 '996415134816980993',
 '896896702971621376',
 '1535062237',
 '1063445437666009088',
 '2422423581',
 '2426316045',
 '425637306',
 '1910925241',
 '2956879863',
 '298129492',
 '311573506',
 '222705540',
 '271816164',
 '921036472639217665',
 '1561423986',
 '475832820',
 '2188688515',
 '475832820',
 '1639176810',
 '280111961',
 '280111961',
 '87970401',
 '328258871',
 '1181646205828419587',
 '305085928',
 '1326971168',
 '2387886752',
 '1692847789',
 '1182104812105674752',
 '836561012',
 '836561012',
 '961823636',
 '127941307',
 '2188688515',
 '1131063617082630144',
 '1106966673922260994',
 '2562308246',
 '775059245771337728',
 '1111558828015972352',
 '1059382959835398144',
 '1146357871321452544',

## EXPLORATORY DATA ANALYSIS AND DATA CLEANING

In [18]:
#shape of the dataframe
df.shape

(59738, 19)

In [19]:
#the first observation from the entities column
df['entities'].iloc[0]

{'hashtags': [{'start': 57, 'end': 63, 'tag': 'Enugu'}],
 'mentions': [{'start': 3, 'end': 15, 'username': 'Maximeks042'}]}

In [20]:
#the first observation from the public_metrics column
df['public_metrics'].iloc[0]

{'retweet_count': 222, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}

In [21]:
#the first observation from the referenced_tweets column
df['referenced_tweets'].iloc[0]

[{'type': 'retweeted', 'id': '1213427724636819457'}]

In [22]:
#getting the keys in the public_metrics dictionary
public_attributes = list(df['public_metrics'].iloc[0].keys())
public_attributes


['retweet_count', 'reply_count', 'like_count', 'quote_count']

## Cleaning the public attribute,reference tweets and entities into separate column

In [10]:
df


,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [65]:
df['public_metrics'].fillna(np.nan,inplace=True)

In [66]:
def get_public_attribute(data_public, attribute):
    if data_public is np.nan: return data_public
    return data_public[attribute]
    
#using for loop to loop over the attributes and apply it to public_metrics column
public_attributes = list(df['public_metrics'].iloc[0].keys())
for public_attribute in public_attributes:
    df['public_metric_' + public_attribute] = df['public_metrics'].apply(get_public_attribute, args = (public_attribute,))


In [67]:
df

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [200]:
df

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [31]:
df['entities'].iloc[0]

{'hashtags': [{'start': 57, 'end': 63, 'tag': 'Enugu'}],
 'mentions': [{'start': 3, 'end': 15, 'username': 'Maximeks042'}]}

In [68]:
df['entities'].fillna(np.nan,inplace=True)

In [69]:
df['entities'].fillna(np.nan,inplace=True)

In [70]:
df

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [144]:
df['id'].dtype

dtype('O')

In [136]:
df.entities.loc[6096]


6096    {'mentions': [{'start': 3, 'end': 16, 'usernam...
6096    {'hashtags': [{'start': 115, 'end': 134, 'tag'...
6096    {'mentions': [{'start': 3, 'end': 17, 'usernam...
6096    {'hashtags': [{'start': 30, 'end': 44, 'tag': ...
6096    {'annotations': [{'start': 70, 'end': 78, 'pro...
6096    {'mentions': [{'start': 3, 'end': 16, 'usernam...
6096    {'urls': [{'start': 30, 'end': 53, 'url': 'htt...
Name: entities, dtype: object

In [146]:
df.query("id=='1321236431575097346'")

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,mentioned_usernames,referenced_type
6096,1321236431575097346,"{'urls': [{'start': 30, 'end': 53, 'url': 'htt...",828923502,2020-10-27T23:44:47.000Z,#endsars\n#Protestpeacefully. \nhttps://t.co/n...,und,False,NaN,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN


In [156]:
queri = df.query("id=='1321236431575097346'")['entities'].iloc[0]
queri.items()


dict_items([('urls', [{'start': 30, 'end': 53, 'url': 'https://t.co/nJ17WTyatk', 'expanded_url': 'https://youtu.be/XUlVkhqg4tI', 'display_url': 'youtu.be/XUlVkhqg4tI', 'images': [{'url': 'https://pbs.twimg.com/news_img/1322143663028359169/Aud0iIwP?format=jpg&name=orig', 'width': 480, 'height': 360}, {'url': 'https://pbs.twimg.com/news_img/1322143663028359169/Aud0iIwP?format=jpg&name=150x150', 'width': 150, 'height': 150}], 'status': 200, 'title': 'leaked: A very Important information from Nigeria Army, please share', 'description': 'leaked: A very Important information from Nigeria Army, please share', 'unwound_url': 'https://www.youtube.com/watch?v=XUlVkhqg4tI&feature=youtu.be'}]), ('hashtags', [{'start': 0, 'end': 8, 'tag': 'endsars'}, {'start': 9, 'end': 27, 'tag': 'Protestpeacefully'}])])

In [158]:
df.entities.iloc[0]
for x,y in queri.items():
    if x=='hashtags':
        for a in y:
            print((a.get('tag')))

endsars
Protestpeacefully


In [164]:
df.entities.loc[6096]


6096    {'mentions': [{'start': 3, 'end': 16, 'usernam...
6096    {'hashtags': [{'start': 115, 'end': 134, 'tag'...
6096    {'mentions': [{'start': 3, 'end': 17, 'usernam...
6096    {'hashtags': [{'start': 30, 'end': 44, 'tag': ...
6096    {'annotations': [{'start': 70, 'end': 78, 'pro...
6096    {'mentions': [{'start': 3, 'end': 16, 'usernam...
6096    {'urls': [{'start': 30, 'end': 53, 'url': 'htt...
Name: entities, dtype: object

In [ ]:
df.entities.iloc[6096]
for x,y in queri.items():
    if x=='hashtags':
        for a in y:
            print((a.get('tag')))

In [13]:
def retrieve_hashtags(data, name, input):
    if type(data) == dict:
        for dic in data.keys():
            if data == input:
                for value in data[dict]:
                    return value.get(name)
            



In [15]:
df.entities.iloc[0]
for x,y in df.entities.iloc[0].items():
    if x=='hashtags':
        for a in y:
            (a.get('tag'))

In [165]:
df.entities.iloc[0]


{'hashtags': [{'start': 57, 'end': 63, 'tag': 'Enugu'}],
 'mentions': [{'start': 3, 'end': 15, 'username': 'Maximeks042'}]}

## HASHTAG

entity has a dictionary as the data structure
inside the dictionary we have list
inside the list we have a dictionary
key is hashtag and mention
while value is []

In [16]:
a_list = []
for keys,values in df.entities.iloc[0].items():
        if keys=='hashtags':
            for a in values:
                a.get('tag')
                a_list.append(a.get('tag'))


In [170]:
a_list

['Enugu']

In [171]:
C

{'hashtags': [{'start': 57, 'end': 63, 'tag': 'Enugu'}],
 'mentions': [{'start': 3, 'end': 15, 'username': 'Maximeks042'}]}

In [71]:
def get_entities(data,first_key,second_key):
    a_list = []
    if type(data) == dict:
        for keys,values in data.items():
            if keys==first_key:
                for a in values:
                    a.get(second_key)
                    a_list.append(a.get(second_key))
                return a_list               

In [72]:
df['hashtags'] = df['entities'].apply(get_entities,first_key='hashtags',second_key='tag')
df['mentions'] = df['entities'].apply(get_entities,first_key='mentions',second_key='username')


In [73]:
df

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [39]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def auth():
    return os.environ.get()




def create_url(users):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values
    user_ids = 'ids='+','.join(users)
    user_fields = "user.fields=description,created_at,id,location,public_metrics,username,verified"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url ="https://api.twitter.com/2/users?{}&{}".format(user_ids, user_fields)
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main(users):
    bearer_token = search_args ['bearer_token']
    url = create_url(users)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    #print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


In [40]:
import time

In [41]:
df['author_id'].dropna().unique().shape

(34758,)

In [45]:
def user_bio_frame(users = list(df['author_id'].dropna().unique())):
    user_count = len(users)
    user_df = pd.DataFrame()
    for iter in range(int(user_count/100)):
        position = iter
        start = iter*100
        end = (iter + 1)*100
        user_bio = main(users = users[start:end])
        user_df = pd.concat([user_df,pd.DataFrame(user_bio['data'])])
        if (position+1)% 300==0:
            pd.to_pickle(user_df,'users_{}.pkl'.format((position+1)/300))
            time.sleep(910)
        else: 
            continue
    return user_df
    

In [46]:
user_profile=user_bio_frame()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [47]:
user_profile

,description,created_at,public_metrics,id,username,verified,name,location
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,NaN
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,NaN
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,NaN
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja
...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,NaN
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁"
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria


In [303]:
user_profile.to_pickle("user_profile.pkl")




In [74]:
profile = pd.read_pickle('user_profile.pkl')

In [75]:
profile.head()

,description,created_at,public_metrics,id,username,verified,name,location
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,NaN
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,NaN
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,NaN
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja


In [25]:
profile.shape

(34696, 8)

In [26]:
profile.columns

Index(['description', 'created_at', 'public_metrics', 'id', 'username',
       'verified', 'name', 'location'],
      dtype='object')

In [31]:
profile['username'].unique()

array(['Johniky5', 'emeriaemmanuel', 'EkpereNwankwo', ..., '_sixile',
       'jumoyeh', 'kiingsirmarrd'], dtype=object)

In [76]:
profile.drop_duplicates(subset="username")


,description,created_at,public_metrics,id,username,verified,name,location
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,NaN
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,NaN
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,NaN
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja
...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,NaN
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁"
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria


In [77]:
profile['location'] = profile['location'].fillna('') # Specific columns

In [78]:
profile['City'] = profile['location'].str.split(',').apply(lambda x: x[0])
profile['Country'] = profile['location'].str.split(',').apply(lambda x: x[-1])


In [79]:
profile

,description,created_at,public_metrics,id,username,verified,name,location,City,Country
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja
...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria


In [80]:
profile

,description,created_at,public_metrics,id,username,verified,name,location,City,Country
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja
...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria


In [42]:
profile

,description,created_at,public_metrics,id,username,verified,name,location,City,Country
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja
...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria


In [81]:
def get_public_metrics(data_public_metrics, metric_attribute):
    if data_public_metrics is np.nan: return data_public_metrics
    return data_public_metrics[metric_attribute]
    
#using for loop to loop over the attributes and apply it to public_metrics column
public_metric_attributes = list(profile['public_metrics'].iloc[0].keys())
for public_metric_attribute in public_metric_attributes:
    profile['public_metric_' + public_metric_attribute] = profile['public_metrics'].apply(get_public_metrics, args = (public_metric_attribute,))
 

In [44]:
profile.shape

(34696, 14)

In [82]:
profile.tail(20)

,description,created_at,public_metrics,id,username,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count
80,"am unique,simple & friendly.",2020-04-16T07:05:12.000Z,"{'followers_count': 17, 'following_count': 67,...",1250681512489607169,mosesbankole12,False,bankole adeoniye,,,,17,67,92,0
81,Your Plug of life🔌 \nApple / Samsung / Tablets...,2015-09-06T13:02:57.000Z,"{'followers_count': 265, 'following_count': 80...",3566420656,Plug_Xnything,False,Just Ask?.....™️,"Ikeja, Nigeria",Ikeja,Nigeria,265,807,1187,0
82,Instagram: Timiturner10,2011-02-14T00:57:51.000Z,"{'followers_count': 1297, 'following_count': 9...",251869380,MsRegularrMimi,False,BITCH IM A MOTHAAAAA,"Everywhere, Anywhere, Nowhere",Everywhere,Nowhere,1297,969,11828,3
83,Fashion designer and an Educator,2012-07-22T02:11:48.000Z,"{'followers_count': 1324, 'following_count': 8...",709922550,Edgeof_desires,False,Crystal Michelle,"Dallas, TX",Dallas,TX,1324,894,45237,8
84,"smile hard,work hard😀😀",2020-02-06T17:14:59.000Z,"{'followers_count': 289, 'following_count': 51...",1225467889802776577,akinloyeayomid4,False,ayo.mide,,,,289,51,1940,0
85,Welcome.\n\n\nThe world is crammed with cruelt...,2018-11-01T22:12:30.000Z,"{'followers_count': 2496, 'following_count': 4...",1058119617883897856,Isnotmynameooo,False,Just Jane🌸🌸🌸#ENDSARS,Paradis Island 🖤,Paradis Island 🖤,Paradis Island 🖤,2496,4258,36946,0
86,I have absolutely no idea what’s meant to be w...,2012-03-16T11:55:52.000Z,"{'followers_count': 938, 'following_count': 85...",526317833,spicykof,False,Riri🌚,"Ibadan, Nigeria",Ibadan,Nigeria,938,850,26449,1
87,💜,2011-07-03T13:52:05.000Z,"{'followers_count': 5272, 'following_count': 9...",328496407,Musee_x,False,🦋,,,,5272,965,172439,15
88,"(MR VALUE) Artist Manager , @JJCskillz @idcab...",2009-01-10T22:20:35.000Z,"{'followers_count': 5395, 'following_count': 3...",18848570,stevespotlight,False,SteveSpotlight,Nigeria/UK,Nigeria/UK,Nigeria/UK,5395,3446,30662,44
89,☥ Everything just is. Just be. ☥ Asè @_IrideSc...,2015-07-15T22:25:34.000Z,"{'followers_count': 289, 'following_count': 65...",3281005862,Classy1k,False,Tt’Shaylah Lightbourn,"Florida, USA",Florida,USA,289,650,2391,1


In [46]:
df

,id,entities,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,public_metrics,geo,...,oldest_id,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,hashtags,mentions
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Enugu],[Maximeks042]
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,NaN,NaN,NaN,NaN,986.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6121,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,1322.0,0.0,0.0,0.0,[EndSARS],[DrOlufunmilayo]
6122,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,2328.0,0.0,0.0,0.0,None,[DrOlufunmilayo]
6123,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,6.0,0.0,3.0,0.0,[endsars],None
6124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [83]:
profile.rename(columns = {'id':'author_id'}, inplace = True)


In [84]:
profile

,description,created_at,public_metrics,author_id,username,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,,6814,5226,406196,44
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,,39,384,2154,0
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,,5827,4772,316438,156
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,867,1372,3505,0
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja,1677,379,31185,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,,18,179,151,0
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....,4438,745,77530,40
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁,7485,968,172460,35
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,817,2778,3996,4


In [85]:
dataf['location'] = dataf['location'].fillna('') # Specific columns

In [86]:
dataf  = df.merge(profile, how = 'left', on = 'author_id')

In [90]:
dataf.columns

Index(['id', 'entities', 'author_id', 'created_at_x', 'text', 'lang',
       'possibly_sensitive', 'referenced_tweets', 'public_metrics_x', 'geo',
       'in_reply_to_user_id', 'users', 'tweets', 'places', 'newest_id',
       'oldest_id', 'result_count', 'next_token', 'withheld',
       'public_metric_retweet_count', 'public_metric_reply_count',
       'public_metric_like_count', 'public_metric_quote_count', 'hashtags',
       'mentions', 'description', 'created_at_y', 'public_metrics_y',
       'username', 'verified', 'name', 'location', 'City', 'Country',
       'public_metric_followers_count', 'public_metric_following_count',
       'public_metric_tweet_count', 'public_metric_listed_count'],
      dtype='object')

In [52]:
dataf.to_pickle("data_combined.pkl")


In [223]:
dataf.location.iloc[3]

'Lagos Nigeria '

In [54]:
dataf['City'] = dataf['location'].str.split(',').apply(lambda x: x[0])
dataf['Country'] = dataf['location'].str.split(',').apply(lambda x: x[-1])


In [56]:
dataf['City'] = dataf['location'].str.split('_').apply(lambda x: x[0])
dataf['Country'] = dataf['location'].str.split('_').apply(lambda x: x[-1])


In [84]:
dataf.head(20)

,id,entities,author_id,created_at_x,text,lang,possibly_sensitive,referenced_tweets,public_metrics_x,geo,...,username,verified,name,location,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count,City,Country
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,Johniky5,False,John iky,,6814.0,5226.0,406196.0,44.0,,
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,emeriaemmanuel,False,Chukwuemeria,,39.0,384.0,2154.0,0.0,,
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,EkpereNwankwo,False,Ekpere Nwankwo,,5827.0,4772.0,316438.0,156.0,,
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,867.0,1372.0,3505.0,0.0,Lagos Nigeria,Lagos Nigeria
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,TheMercyWhite,False,Ify,Abuja,1677.0,379.0,31185.0,9.0,Abuja,Abuja
5,1214624772987990021,NaN,1014823661063344129,2020-01-07T19:08:07.000Z,RT @Chiileetaa: It's SPECIAL ANTI-ROBBERY SQUA...,en,False,"[{'type': 'retweeted', 'id': '1214521047636164...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,...,_igwilo,False,K3lechi †,Not in a good mood,248210.0,13032.0,226308.0,150.0,Not in a good mood,Not in a good mood
6,1214605164826562560,"{'hashtags': [{'start': 64, 'end': 72, 'tag': ...",326132860,2020-01-07T17:50:12.000Z,With Faces Backing The Camera Unlike How Civil...,en,False,"[{'type': 'quoted', 'id': '1214488973038497792'}]","{'retweet_count': 1, 'reply_count': 0, 'like_c...",{'place_id': '014326ae7dbb50ee'},...,oladaybak,False,🦍 mR. PaLaSh 🔺,eVeRyWhErE PaLaSh ✈,7524.0,7308.0,6781.0,0.0,eVeRyWhErE PaLaSh ✈,eVeRyWhErE PaLaSh ✈
7,1214602210123763712,"{'annotations': [{'start': 106, 'end': 114, 'p...",1066054522517114880,2020-01-07T17:38:27.000Z,RT @CivicHive: We urge @PoliceNG to look into ...,en,False,"[{'type': 'retweeted', 'id': '1214099894534836...","{'retweet_count': 12, 'reply_count': 0, 'like_...",NaN,...,wallofshame777,False,Wallofshame,,523.0,1841.0,2487.0,0.0,,
8,1214588077303357447,"{'mentions': [{'start': 0, 'end': 14, 'usernam...",398584109,2020-01-07T16:42:18.000Z,@switbeckssexy Lekki and ikoyi link bridge\nAm...,en,False,"[{'type': 'replied_to', 'id': '121458754060017...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,...,JeromeOgunna,False,Jerome,"lagos, Nigeria",636.0,485.0,10956.0,0.0,"lagos, Nigeria","lagos, Nigeria"
9,1214584687219359744,"{'annotations': [{'start': 106, 'end': 114, 'p...",2232504074,2020-01-07T16:28:49.000Z,RT @CivicHive: We urge @PoliceNG to look into ...,en,False,"[{'type': 'retweeted', 'id': '1214099894534836...","{'retweet_count': 12, 'reply_count': 0, 'like_...",NaN,...,olusola_adebomi,False,Olusola Adebomi,,147.0,594.0,7713.0,0.0,,


In [328]:
df.head()

,id,author_id,created_at,text,lang,possibly_sensitive,referenced_tweets,geo,in_reply_to_user_id,users,...,result_count,next_token,withheld,public_metric_retweet_count,public_metric_reply_count,public_metric_like_count,public_metric_quote_count,mentioned_usernames,hashtags,mentions
0,1214674154991042560,782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...",NaN,NaN,NaN,...,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Maximeks042],[Enugu],[Maximeks042]
1,1214667391206141957,827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...",NaN,NaN,NaN,...,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Maximeks042],[Enugu],[Maximeks042]
2,1214648097919447040,3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...",NaN,NaN,NaN,...,NaN,NaN,NaN,222.0,0.0,0.0,0.0,[Maximeks042],[Enugu],[Maximeks042]
3,1214638748580884480,954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...",NaN,NaN,NaN,...,NaN,NaN,NaN,986.0,0.0,0.0,0.0,[DrOlufunmilayo],None,[DrOlufunmilayo]
4,1214629356858937344,313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,None,None


In [ ]:
new_data = pd.merge(df_row, df3, left_on='id', right_on='id')


In [ ]:
def get_entities(data,first_key,second_key):
    a_list = []
    if type(data) == dict:
        for keys,values in data.items():
            if keys==first_key:
                for a in values:
                    a.get(second_key)
                    a_list.append(a.get(second_key))
                return a_list               

In [ ]:
str = 'Hello! I am Robot. This is a Python example.'

#split string
splits = str.split()

#for loop to iterate over words array
for split in splits:
	print(split)

In [66]:
import pycountry


In [138]:
import re
location= 'south Nigeria'
str = re.split(r' |,|\.', location)
for country in pycountry.countries:
    if country.name in str:
        print(country.name)


Nigeria


In [ ]:
def get_location(data):
    a_list = []
        for a in pycountry.data:
            if data.name in text:
                a_list.append(a.get(second_key))
                return a_list               

In [ ]:
import pycountry
for element in string_name:


## Cleaning the location

In [ ]:
import re



In [ ]:
txt = profile['location']
x = re.search("^The.*Spain$", txt)


In [ ]:
def extract_Nigeria_location:
    if 

In [63]:
profile['location'].describe()

count              27019
unique              9904
top       Lagos, Nigeria
freq                4298
Name: location, dtype: object

In [67]:
profile['location'].describe()

dtype('O')

In [115]:
dataf

,id,entities,author_id,created_at_x,text,lang,possibly_sensitive,referenced_tweets,public_metrics_x,geo,...,username,verified,name,location,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count,City,Country
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,Johniky5,False,John iky,,6814.0,5226.0,406196.0,44.0,,
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,emeriaemmanuel,False,Chukwuemeria,,39.0,384.0,2154.0,0.0,,
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,EkpereNwankwo,False,Ekpere Nwankwo,,5827.0,4772.0,316438.0,156.0,,
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,867.0,1372.0,3505.0,0.0,Lagos Nigeria,Lagos Nigeria
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,TheMercyWhite,False,Ify,Abuja,1677.0,379.0,31185.0,9.0,Abuja,Abuja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59733,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,HoneyLaycon,False,Laycon's HONEY💡💡💡,,144.0,620.0,512.0,0.0,,
59734,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,
59735,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,
59736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,


In [168]:
dataf["location"]= dataf["location"].str.upper().str.title()


In [169]:
dataf

,id,entities,author_id,created_at_x,text,lang,possibly_sensitive,referenced_tweets,public_metrics_x,geo,...,username,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,Johniky5,False,John iky,,,,6814.0,5226.0,406196.0,44.0
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,emeriaemmanuel,False,Chukwuemeria,,,,39.0,384.0,2154.0,0.0
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,EkpereNwankwo,False,Ekpere Nwankwo,,,,5827.0,4772.0,316438.0,156.0
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,867.0,1372.0,3505.0,0.0
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja,1677.0,379.0,31185.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59733,1321236418228879360,"{'hashtags': [{'start': 114, 'end': 122, 'tag'...",1301265955335671811,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: The Army is saying they we...,en,False,"[{'type': 'retweeted', 'id': '1321211738088476...","{'retweet_count': 1322, 'reply_count': 0, 'lik...",NaN,...,HoneyLaycon,False,Laycon's HONEY💡💡💡,,,,144.0,620.0,512.0,0.0
59734,1321236418023313408,"{'annotations': [{'start': 65, 'end': 79, 'pro...",1042117744672550914,2020-10-27T23:44:44.000Z,RT @DrOlufunmilayo: Guys if you want any justi...,en,False,"[{'type': 'retweeted', 'id': '1321215486600859...","{'retweet_count': 2328, 'reply_count': 0, 'lik...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59735,1321236417717174272,"{'hashtags': [{'start': 13, 'end': 21, 'tag': ...",456777070,2020-10-27T23:44:44.000Z,Never Forget #endsars https://t.co/CWpb8wLHXI,en,False,NaN,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
def get_country_code(data):
    if type(data) == str:
        for i in data.split():
            return pycountry.countries
dataf['Country_new'] = dataf['location'].apply(get_country_code)


In [185]:
dataf.head(10)

,id,entities,author_id,created_at_x,text,lang,possibly_sensitive,referenced_tweets,public_metrics_x,geo,...,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count,Country_new
0,1214674154991042560,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",782957256434409472,2020-01-07T22:24:20.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,False,John iky,,,,6814.0,5226.0,406196.0,44.0,None
1,1214667391206141957,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",827863696416321537,2020-01-07T21:57:28.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,False,Chukwuemeria,,,,39.0,384.0,2154.0,0.0,None
2,1214648097919447040,"{'hashtags': [{'start': 57, 'end': 63, 'tag': ...",3344738026,2020-01-07T20:40:48.000Z,RT @Maximeks042: Today I witnesses brutality i...,en,False,"[{'type': 'retweeted', 'id': '1213427724636819...","{'retweet_count': 222, 'reply_count': 0, 'like...",NaN,...,False,Ekpere Nwankwo,,,,5827.0,4772.0,316438.0,156.0,None
3,1214638748580884480,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",954711989842010112,2020-01-07T20:03:39.000Z,"RT @DrOlufunmilayo: VERY URGENT:\nSince 2017, ...",en,False,"[{'type': 'retweeted', 'id': '1205787583000829...","{'retweet_count': 986, 'reply_count': 0, 'like...",NaN,...,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,867.0,1372.0,3505.0,0.0,None
4,1214629356858937344,"{'annotations': [{'start': 134, 'end': 144, 'p...",313725796,2020-01-07T19:26:20.000Z,So after they made people’s parents wear costu...,en,False,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,...,False,Ify,Abuja,Abuja,Abuja,1677.0,379.0,31185.0,9.0,None
5,1214624772987990021,NaN,1014823661063344129,2020-01-07T19:08:07.000Z,RT @Chiileetaa: It's SPECIAL ANTI-ROBBERY SQUA...,en,False,"[{'type': 'retweeted', 'id': '1214521047636164...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,...,False,K3lechi †,Not In A Good Mood,Not in a good mood,Not in a good mood,248210.0,13032.0,226308.0,150.0,None
6,1214605164826562560,"{'hashtags': [{'start': 64, 'end': 72, 'tag': ...",326132860,2020-01-07T17:50:12.000Z,With Faces Backing The Camera Unlike How Civil...,en,False,"[{'type': 'quoted', 'id': '1214488973038497792'}]","{'retweet_count': 1, 'reply_count': 0, 'like_c...",{'place_id': '014326ae7dbb50ee'},...,False,🦍 mR. PaLaSh 🔺,Everywhere Palash ✈,eVeRyWhErE PaLaSh ✈,eVeRyWhErE PaLaSh ✈,7524.0,7308.0,6781.0,0.0,None
7,1214602210123763712,"{'annotations': [{'start': 106, 'end': 114, 'p...",1066054522517114880,2020-01-07T17:38:27.000Z,RT @CivicHive: We urge @PoliceNG to look into ...,en,False,"[{'type': 'retweeted', 'id': '1214099894534836...","{'retweet_count': 12, 'reply_count': 0, 'like_...",NaN,...,False,Wallofshame,,,,523.0,1841.0,2487.0,0.0,None
8,1214588077303357447,"{'mentions': [{'start': 0, 'end': 14, 'usernam...",398584109,2020-01-07T16:42:18.000Z,@switbeckssexy Lekki and ikoyi link bridge\nAm...,en,False,"[{'type': 'replied_to', 'id': '121458754060017...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,...,False,Jerome,"Lagos, Nigeria",lagos,Nigeria,636.0,485.0,10956.0,0.0,None
9,1214584687219359744,"{'annotations': [{'start': 106, 'end': 114, 'p...",2232504074,2020-01-07T16:28:49.000Z,RT @CivicHive: We urge @PoliceNG to look into ...,en,False,"[{'type': 'retweeted', 'id': '1214099894534836...","{'retweet_count': 12, 'reply_count': 0, 'like_...",NaN,...,False,Olusola Adebomi,,,,147.0,594.0,7713.0,0.0,None


In [93]:
dataf['tweets'].T

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
59733                                                  NaN
59734                                                  NaN
59735                                                  NaN
59736    [{'text': 'Took this picture at 5:45 PM last T...
59737                                                  NaN
Name: tweets, Length: 59738, dtype: object

In [225]:
profile

,description,created_at,public_metrics,author_id,username,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,,6814,5226,406196,44
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,,39,384,2154,0
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,,5827,4772,316438,156
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,867,1372,3505,0
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja,1677,379,31185,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,,18,179,151,0
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....,4438,745,77530,40
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁,7485,968,172460,35
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,817,2778,3996,4


In [226]:
def get_country_code(data):
    if type(data) == str:
        for i in data.split():
            return pycountry.countries
profile['Country_new'] = profile['location'].apply(get_country_code)


In [54]:
profile

,description,created_at,public_metrics,author_id,username,verified,name,location,City,Country,public_metric_followers_count,public_metric_following_count,public_metric_tweet_count,public_metric_listed_count
0,,2016-10-03T14:55:31.000Z,"{'followers_count': 6814, 'following_count': 5...",782957256434409472,Johniky5,False,John iky,,,,6814,5226,406196,44
1,,2017-02-04T12:57:41.000Z,"{'followers_count': 39, 'following_count': 384...",827863696416321537,emeriaemmanuel,False,Chukwuemeria,,,,39,384,2154,0
2,,2015-06-24T22:38:45.000Z,"{'followers_count': 5827, 'following_count': 4...",3344738026,EkpereNwankwo,False,Ekpere Nwankwo,,,,5827,4772,316438,156
3,Legal Practitioner Fashi...,2018-01-20T13:47:30.000Z,"{'followers_count': 867, 'following_count': 13...",954711989842010112,ALegalEssence,False,Your TaiLaw,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,867,1372,3505,0
4,Égalité | Don’t stain my white. 📩themercywhite...,2011-06-09T03:33:04.000Z,"{'followers_count': 1677, 'following_count': 3...",313725796,TheMercyWhite,False,Ify,Abuja,Abuja,Abuja,1677,379,31185,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Easy going, optimist,foodie😍😍\nA goal getter ❤",2020-10-11T02:42:40.000Z,"{'followers_count': 18, 'following_count': 179...",1315120514961223682,AwonugaDammy,False,limat,,,,18,179,151,0
96,Creative Director - @InkedPresses• Founder and...,2010-11-06T17:41:52.000Z,"{'followers_count': 4438, 'following_count': 7...",212655875,asanibijoux,False,Lola makes the Best Pressons,a miracle in the making....,a miracle in the making....,a miracle in the making....,4438,745,77530,40
97,the muse. . . a visual intoxication. . . fire ...,2009-09-03T18:22:29.000Z,"{'followers_count': 7485, 'following_count': 9...",71321343,_sixile,False,dr. angelface.,"TMC, TX 💙🏁",TMC,TX 💙🏁,7485,968,172460,35
98,"founder @hafiorganics,❤ Naija ❤ books,❤ food,\...",2013-06-01T04:19:19.000Z,"{'followers_count': 817, 'following_count': 27...",1473768728,jumoyeh,False,'Lajumoke Omo Oye,Lagos Nigeria,Lagos Nigeria,Lagos Nigeria,817,2778,3996,4


In [186]:
def get_country(data):
    if type(data) == str:
        for i in data.split():
            return pycountry.countries
dataf['Country_new'] = dataf['location'].apply(get_country_code)





import re
location= 'south Nigeria'.title()
str = re.split(r' |,|\.', location)
for country in pycountry.countries:
    if country.name in str:
        print(country.name)


Nigeria


In [ ]:
def get_dt_after(start_date, delta=timedelta(days=7)):
    """
    Create a function where you put in a date and it returns a date 7 days after in string format
    """

In [ ]:
def create_tweets_df():
    Create a function that Creates a dataframe from the tweets you've retrieved using the retieve_tweets function 
    you created above. This functions should also take a list of datest you want to get tweets from
    """


## Fetch and Clean Data

Use the functions you've created above to collect all the tweets

In [ ]:
dates_list = ["2020-12-01",  "2020-12-08"]

In [ ]:
tweets_df = create_tweets_df(dates_list, max_tweets=10000)
tweets_df.shape

Make sure to save the tweets after this process, use a file format that won't comprimise the characters in tweets

In [ ]:
def delete_duplicates(df):
    """
    Create a function that looks at the tweets in the tweets dataframe and other characters and delete 
    duplicates as sometimes we can get duplicate data/tweets. For example if I tweet something and someone else
    retweets it then this could possibly be a duplicate. Is this useful information? You need to decide of course.
    """

def preprocess_tweet(tweet):
    """
    Create a function that preprocess the text in a single tweet. Suggestions for this function are:
    - correct all multiple white spaces to a single white space
    - convert all urls to string "URL"
    - if the username isn't relevant then convert @username to "AT_USER"
    - converts emoji's to text description of the emoji
    """

In [ ]:
tweets_df = delete_duplicates(tweets_to_analyse)
tweets_df.reset_index(inplace=True, drop=True)
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess_tweet)

## Alt data

In [ ]:
df = pd.read_excel('https://query.data.world/s/rghc6hq5b32ertwgebgwuyx2lcpxvt')

## Tweets Analysis

Now it's time to analyse this data to try and get some interesting insights. Below are some questions you can answer to get started but the best Data Scientists are creative thinkers so think out of th ebox to decide what will be interesting insight

**What locations in Nigeria and around the world were the hashtags popular?**

**How far did the hashtag reach and how deep was this reach?**

**When the did movement start on social media? When did it reach critical mass?**

**What were the most popular words or terms used during the movement?**

**Who were the key influences and proponents involved in the movement?**

**What was the most popular tweet?**

**What is the general sentiment of the tweets about the movement? How has this changed from the start of the movement till now?**

**Were there any bots or bad actors spreading fake news around the movement?**

**What are some examples of this fake news?**

## Network Analysis

Using networkx or any other network graph library, create a social network to identifiy influential people of the movement

## Sentiment Modelling
Train model on labelled twitter sentiment data from another dataset
<br>Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to endsars tweets text

### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [ ]:
def prep_data(df, filename=""):
    """
    Create a functions that prepares the endsars data in a format that the fasttext model likes
    """

**Train Model**

**Apply Modelto Test data**

**Score Model**

**Check labels on example tweets**

### Apply trained fastText model to tweets df

In [ ]:
tweets_df['ft_predict'] = tweets_df.text.apply(ft_model.predict)
tweets_df["sentiment"] = tweets_df.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_df['sentiment_prob'] = tweets_df.ft_predict.apply(lambda x: x[1][0])

In [ ]:
def print_tweet_sentiment(index=0):
    """
    Create function where you either pass the index of the tweet in the tweets_df or a tweet and it returns the 
    sentiment for the tweet
    """